In [10]:
import sys
import requests
import json
import pandas as pd
from datetime import datetime, timedelta
import os
import shutil

from pprint import PrettyPrinter
pp = PrettyPrinter(indent=4)



In [11]:
def adjust_sensors_delay_time(implementation):
    """
    This function returns the delay time for each sensor in order to adjust the data
    """
    #esp32
    seconds_delay = 0
    milliseconds_delay = 0
    if implementation == "ESP32S3": 
        seconds_delay = 2
        milliseconds_delay = 920

    return timedelta(seconds=seconds_delay, milliseconds=milliseconds_delay)

In [12]:
def jobs_unix_timestamp(implementation, day, month):
    epoch = datetime.utcfromtimestamp(0)

    #startDT = datetime(2023, month, 13, 19, 0, 0, 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    startDT = datetime(2023, month, day, 9, 0, 0, 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    startTS = int((startDT - epoch).total_seconds() * 1000.0)
    #endDT = datetime(2023, month, 14, 7, 0, 0 , 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    endDT = datetime(2023, month, day, 18, 0, 0 , 0) - timedelta(hours=1) + adjust_sensors_delay_time(implementation)
    endTS = int((endDT - epoch).total_seconds() * 1000.0)
    

    return str(startTS), str(endTS)

In [13]:
RFID_analysis_keys = [
    'gabarit'
]

TASK_analysis_keys = [
    'task', 'empid', 'sec'
]


TASKDISTANCE_analysis_keys= [
    'distance',
]
BLE_analysis_keys = [
    'id','rssi'
]



In [14]:
def json_csv(data, dataset, implementation):
    data_df = pd.DataFrame()
    first_flag = True
    
    if dataset == "rfid":
        analysis_keys = RFID_analysis_keys
    elif dataset == "task":
        analysis_keys = TASK_analysis_keys
    elif dataset == "taskdistance":
        analysis_keys = TASKDISTANCE_analysis_keys
    else:
        analysis_keys = BLE_analysis_keys



    for analysis_key in analysis_keys:
        if first_flag:
            data_df = pd.DataFrame(data[analysis_key])
            data_df = data_df.rename(columns={'value': analysis_key})
            first_flag = False
        else:
            data_df = data_df.join(pd.DataFrame(data[analysis_key]).set_index('ts'), on='ts')
            data_df = data_df.rename(columns={'value': analysis_key})
    # change the timestamp to seconds
    data_df['ts'] = pd.to_datetime(data_df['ts']/1000,unit='s') + timedelta(hours=1) - adjust_sensors_delay_time(implementation)
    data_df['ts']= data_df['ts'].astype('datetime64[s]')
    return data_df.iloc[::-1]

def getToken():
    username = 'isabel.portugal@tecnico.ulisboa.pt' #email used to create your ThingsBoard account (your email associated with tennant)
    password = 'tHn%Mv1pnmq2av='		#password of your ThingsBoard account (or your tennant account)
    url = 'http://iot.istartlab.tecnico.ulisboa.pt/api/auth/login'
    headers = {'Content-Type': 'application/json', 'Accept': 'application/json'}
    loginJSON = {'username': username, 'password': password}
    tokenAuthResp = requests.post(url, headers=headers, json=loginJSON).json()
    #print(tokenAuthResp)
    token = tokenAuthResp['token']
    return token

In [15]:
tkn = getToken()

my_headers = {'X-Authorization':  "Bearer " + tkn, "Content-Type" : "application/json"}
#print(my_headers)
#url to test connection
url = "http://iot.istartlab.tecnico.ulisboa.pt/api/plugins/telemetry/DEVICE/27110c40-2a6b-11ee-9b43-67453a5ac630/keys/timeseries"

response = requests.get(url, headers=my_headers)
print(response) , print(response.text)

my_headers = {'X-Authorization':  "Bearer " + tkn, "Content-Type" : "application/json"}
#print(my_headers)

<Response [404]>
Device with requested id wasn't found!


In [16]:

def get_response(day_int,month_int,dataset):

    day_str = str(day_int).zfill(2)
    month_str = str(month_int).zfill(2)
    date = day_str + "_" + month_str
    print(date)

    startTS, endTS = jobs_unix_timestamp("ESP32S3", day_int, month_int)

    limit = "1000000000"

    date_filename = dataset + "_" + day_str + "_" + month_str

    cnt = 1

    if dataset == "rfid":
        devices = ["27110c40-2a6b-11ee-9b43-67453a5ac633","f4765800-2cc5-11ee-a22d-b5d6c11ab119","10fc9700-2cc6-11ee-a22d-b5d6c11ab119"]
        #for analysis_key in RFID_analysis_keys:
        keys = "ts,gabarit"

    elif dataset == "task":
        devices = ["502ba360-d205-11ed-a394-2dc30865e203"]
        #for analysis_key in TASK_analysis_keys:
        keys = "task,empid,sec" 
        
    elif dataset == "taskdistance":
        devices = ["502ba360-d205-11ed-a394-2dc30865e203"]
        keys = "distance"

    elif dataset == "ble":
        devices = ["8516caf0-7a71-11ed-a394-2dc30865e203","fe7d6ff0-b52e-11ed-a394-2dc30865e203","4b593330-2498-11ee-a394-2dc30865e203" ]
        #1: fim da linha, 2: pacotes, 3: elevator
        #for analysis_key in BLE_analysis_keys:
        keys = "id,rssi"
    else:   
        print("Dataset not found")

    print(keys)

    for dev in devices:
        url = "http://iot.istartlab.tecnico.ulisboa.pt/api/plugins/telemetry/DEVICE/" + dev + "/values/timeseries?keys=" + keys + "&startTs=" + startTS + "&endTs=" + endTS + "&limit=" + limit
        response = requests.get(url, headers=my_headers)
        print(response.text)
        #if response.text:
        #    print("No data found in that day")
        #    return
        if len(response.text) == 2:
            print("No data found in that day")
            return
        print("Response received", response.text)
        data = json.loads(response.text)
        data_df = json_csv(data, dataset, "ESP32S3")


        if dataset == 'ble':
            parent_dir = 'C:/Users/isacc/Documents/Faculdade/5ano/Tese/Codigo/API/'
            directory = 'ble_' + date
            path = os.path.join(parent_dir, directory) 
            if not os.path.exists(path):
                os.mkdir(path)
            
        print("CNT", cnt)

        if dataset == "rfid":
            data_df.to_csv(date_filename + "_" + str(cnt) + ".csv")

        elif dataset == "ble":
            #base_folder = date_filenam
            base_folder = 'ble_' + date
            file_name = date_filename + "_" + str(cnt) + ".csv"
            file_path = os.path.join(base_folder, file_name)
            print(file_path)
            data_df.to_csv(file_path)

        else:
            data_df.to_csv(date_filename + ".csv")
                    #
        cnt += 1

    print("done")
    return

In [21]:
# CHANGE ONLY THIS
#31.07 ate 30.09
dataset = "ble"#"task"

if dataset == "taskdistance" or dataset == "ble" or dataset == "rfid":
    start_date = '11/10/2023'
    end_date = '11/10/2023'
    start_date = pd.to_datetime(start_date, format='%d/%m/%Y')
    end_date = pd.to_datetime(end_date, format='%d/%m/%Y')
    date_range = pd.date_range(start=start_date, end=end_date)
    for date in date_range:
        day_int = date.day
        month_int = date.month
        get_response(day_int,month_int,dataset)
        print()
else:
    day_int = 13
    month_int = 10
    get_response(day_int,month_int,dataset)

11_10
id,rssi
{"id":[{"ts":1697043590988,"value":"1"},{"ts":1697043555006,"value":"1"},{"ts":1697043548964,"value":"1"},{"ts":1697043536985,"value":"1"},{"ts":1697043494922,"value":"2"},{"ts":1697043488957,"value":"2"},{"ts":1697043476906,"value":"2"},{"ts":1697043464907,"value":"2"},{"ts":1697043464899,"value":"1"},{"ts":1697043452912,"value":"2"},{"ts":1697043440932,"value":"2"},{"ts":1697043440879,"value":"1"},{"ts":1697043428875,"value":"2"},{"ts":1697043416880,"value":"1"},{"ts":1697043404887,"value":"1"},{"ts":1697043398853,"value":"2"},{"ts":1697043392856,"value":"2"},{"ts":1697043392850,"value":"1"},{"ts":1697043386863,"value":"1"},{"ts":1697043374829,"value":"2"},{"ts":1697043368844,"value":"1"},{"ts":1697043362818,"value":"2"},{"ts":1697043356860,"value":"2"},{"ts":1697043344805,"value":"1"},{"ts":1697043314789,"value":"2"},{"ts":1697043308835,"value":"2"},{"ts":1697043308783,"value":"1"},{"ts":1697043302792,"value":"2"},{"ts":1697043296772,"value":"1"},{"ts":1697043290812,"v

In [18]:
print(response.text)

Device with requested id wasn't found!
